In [1]:
import pandas as pd
import numpy as np

import pymysql
import sqlalchemy as db

import getpass 

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def diff_s(seriesA, seriesB):
    setA = set(seriesA)
    setB = set(seriesB)
    return setA-setB

## World Bank indicators per year for each country

### Available indicators https://data.worldbank.org/indicator

### Source https://datacatalog.worldbank.org/dataset/world-development-indicators 
### and http://databank.worldbank.org/data/download/WDI_csv.zip
WDIData.csv

### Data collection

In [4]:
wdi = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\inputs\WDIData.csv')

In [5]:
# verify the dataframe size before processing
wdi.shape

(377784, 65)

### Data cleaning and preprocessing

In [6]:
# select some indicators
indicators = ['SP.POP.TOTL','SE.XPD.CTOT.ZS','IT.NET.BBND.P2','NY.GNP.PCAP.CD','SI.DST.04TH.20','SI.DST.05TH.20', 
              'SI.DST.FRST.20','SI.DST.02ND.20','SI.DST.03RD.20','IT.NET.USER.P2','IT.CEL.SETS.P2', 
              'SI.POV.NAGP','SI.POV.NAHC','SL.UEM.TOTL.NE.ZS','SL.UEM.TOTL.ZS','SL.UEM.ADVN.ZS', 
              'SL.UEM.BASC.ZS','SL.UEM.INTM.ZS','SL.UEM.1524.ZS','SL.UEM.1524.NE.ZS']

# keep only the countries
excluded_countries = ['Arab World', 'Caribbean small states',
       'Central Europe and the Baltics', 'Early-demographic dividend',
       'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)',
       'East Asia & Pacific (IDA & IBRD countries)', 'East Asia & Pacific (IDA & IBRD)',
        'Euro area','Europe & Central Asia',
       'Europe & Central Asia (excluding high income)',
       'Europe & Central Asia (IDA & IBRD countries)', 'Europe & Central Asia (IDA & IBRD)',
        'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only',
       'IDA total', 'Late-demographic dividend',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & the Caribbean (IDA & IBRD countries)',
        'Latin America & the Caribbean (IDA & IBRD)',
        'Latin America & Caribbean (IDA & IBRD)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle income',
       'Middle East & North Africa',
       'Middle East & North Africa (excluding high income)',
       'Middle East & North Africa (IDA & IBRD countries)',
        'Middle East & North Africa (IDA & IBRD)',
       'Middle income', 'North America', 'Not classified', 'OECD members',
       'Other small states', 'Pacific island small states',
       'Post-demographic dividend', 'Pre-demographic dividend',
       'Small states', 'South Asia', 'South Asia (IDA & IBRD)',
       'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
       'Sub-Saharan Africa (IDA & IBRD countries)', 
        'Sub-Saharan Africa (IDA & IBRD)','Upper middle income',
       'World']

In [7]:
# removes extra spaces in column names
wdi.columns = wdi.columns.str.strip().str.replace(' ', '').str.replace('_', '').str.replace('(', '').str.replace(')', '')

In [8]:
# select indicators and countries
wdi = wdi[wdi["IndicatorCode"].isin(indicators) & ~wdi["CountryName"].isin(excluded_countries)]

In [9]:
# select the year range et the columns
factors = ['2015', '2016', '2017', '2018', '2019']
cols_wdi = ['CountryName', 'CountryCode', 'IndicatorName', 'IndicatorCode']

In [10]:
# transform the years columns in rows
wdi = pd.melt(wdi, id_vars=cols_wdi, value_vars=factors)

In [11]:
# rename the columns
mapping = {'variable': 'Years', 'value': 'Val'}
wdi = wdi.rename(mapping, axis=1)

In [12]:
wdi.shape

(20615, 6)

In [13]:
wdi.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Years,Val
0,Afghanistan,AFG,"Current education expenditure, total (% of tot...",SE.XPD.CTOT.ZS,2015,80.407330
1,Afghanistan,AFG,Fixed broadband subscriptions (per 100 people),IT.NET.BBND.P2,2015,0.020535
2,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2015,600.000000
3,Afghanistan,AFG,Income share held by fourth 20%,SI.DST.04TH.20,2015,NaN
4,Afghanistan,AFG,Income share held by highest 20%,SI.DST.05TH.20,2015,NaN


In [14]:
wdi.tail()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Years,Val
20610,Zimbabwe,ZWE,Unemployment with intermediate education (% of...,SL.UEM.INTM.ZS,2019,NaN
20611,Zimbabwe,ZWE,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,2019,4.898
20612,Zimbabwe,ZWE,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.NE.ZS,2019,NaN
20613,Zimbabwe,ZWE,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.ZS,2019,8.132
20614,Zimbabwe,ZWE,"Unemployment, youth total (% of total labor fo...",SL.UEM.1524.NE.ZS,2019,NaN


## World Bank indicators by countries

### Source https://datacatalog.worldbank.org/dataset/world-development-indicators 
### and http://databank.worldbank.org/data/download/WDI_csv.zip
WDICountry.csv

### Data collection

In [15]:
wdc = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\inputs\WDICountry.csv')

In [16]:
# verify the dataframe size
wdc.shape

(263, 31)

### Data cleaning and preprocessing

In [17]:
# removes extra spaces in column names
wdc.columns = wdc.columns.str.strip().str.replace(' ', '').str.replace('_', '').str.replace('(', '').str.replace(')', '')

In [18]:
wdc.rename(columns={'ShortName':'Country'}, inplace=True)

In [19]:
# select the countries
wdc = wdc[~wdc["Country"].isin(excluded_countries)]

In [20]:
# select the columns
wdc = wdc[['CountryCode', 'Country', 'Region', 'IncomeGroup']]

### Measuring and optimising

In [21]:
# check the values of the column to declare it in ENUM when optimizing the DB
wdc['IncomeGroup'].value_counts()

High income            79
Upper middle income    60
Lower middle income    47
Low income             31
Name: IncomeGroup, dtype: int64

In [22]:
# check the values of the column to declare it in ENUM when optimizing the DB
wdc['Region'].value_counts()

Europe & Central Asia         58
Sub-Saharan Africa            48
Latin America & Caribbean     42
East Asia & Pacific           37
Middle East & North Africa    21
South Asia                     8
North America                  3
Name: Region, dtype: int64

In [23]:
# get ready for a jointure with World Happiness
wdc['Country'] = wdc['Country'].str.lower()

## Import the OECD file in order to identify the OECD countries with the ones in the DF

### Source https://data.oecd.org/natincome/net-national-income.htm
### OECD - OECD countries national net income 2014 - 2018 

OECD_NNI.csv

### Data collection

In [24]:
oecd = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\inputs\OECD_NNI.csv')

In [25]:
oecd_list = oecd['LOCATION'].unique()

In [ ]:
# add a boolean type column
wdc['OECD'] = wdc['CountryCode'].isin(oecd_list)

In [27]:
# verify the dataframe size
wdc.shape

(217, 5)

In [28]:
wdc.head()

,CountryCode,Country,Region,IncomeGroup,OECD
0,ABW,aruba,Latin America & Caribbean,High income,False
1,AFG,afghanistan,South Asia,Low income,False
2,AGO,angola,Sub-Saharan Africa,Lower middle income,False
3,ALB,albania,Europe & Central Asia,Upper middle income,False
4,AND,andorra,Europe & Central Asia,High income,False


In [29]:
wdc.tail()

,CountryCode,Country,Region,IncomeGroup,OECD
258,XKX,kosovo,Europe & Central Asia,Upper middle income,False
259,YEM,yemen,Middle East & North Africa,Low income,False
260,ZAF,south africa,Sub-Saharan Africa,Upper middle income,False
261,ZMB,zambia,Sub-Saharan Africa,Lower middle income,False
262,ZWE,zimbabwe,Sub-Saharan Africa,Lower middle income,False


## World Happiness Report data https://worldhappiness.report/

### Choice : the data available on Kaggle from 2015 to 2019
### Source https://www.kaggle.com/unsdsn/world-happiness  

### Data collection

In [30]:
wh15 = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\inputs\2015.csv')
wh16 = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\inputs\2016.csv')
wh17 = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\inputs\2017.csv')
wh18 = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\inputs\2018.csv')
wh19 = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\inputs\2019.csv')

In [31]:
# check the DF characteristics
wh15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 12 columns):
Country                          158 non-null object
Region                           158 non-null object
Happiness Rank                   158 non-null int64
Happiness Score                  158 non-null float64
Standard Error                   158 non-null float64
Economy (GDP per Capita)         158 non-null float64
Family                           158 non-null float64
Health (Life Expectancy)         158 non-null float64
Freedom                          158 non-null float64
Trust (Government Corruption)    158 non-null float64
Generosity                       158 non-null float64
Dystopia Residual                158 non-null float64
dtypes: float64(9), int64(1), object(2)
memory usage: 14.9+ KB


In [32]:
# check the DF characteristics
wh17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 12 columns):
Country                          155 non-null object
Happiness.Rank                   155 non-null int64
Happiness.Score                  155 non-null float64
Whisker.high                     155 non-null float64
Whisker.low                      155 non-null float64
Economy..GDP.per.Capita.         155 non-null float64
Family                           155 non-null float64
Health..Life.Expectancy.         155 non-null float64
Freedom                          155 non-null float64
Generosity                       155 non-null float64
Trust..Government.Corruption.    155 non-null float64
Dystopia.Residual                155 non-null float64
dtypes: float64(10), int64(1), object(1)
memory usage: 14.6+ KB


In [33]:
# check the DF characteristics
wh18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 9 columns):
Overall rank                    156 non-null int64
Country or region               156 non-null object
Score                           156 non-null float64
GDP per capita                  156 non-null float64
Social support                  156 non-null float64
Healthy life expectancy         156 non-null float64
Freedom to make life choices    156 non-null float64
Generosity                      156 non-null float64
Perceptions of corruption       155 non-null float64
dtypes: float64(7), int64(1), object(1)
memory usage: 11.0+ KB


In [34]:
# check the DF characteristics
wh19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 9 columns):
Overall rank                    156 non-null int64
Country or region               156 non-null object
Score                           156 non-null float64
GDP per capita                  156 non-null float64
Social support                  156 non-null float64
Healthy life expectancy         156 non-null float64
Freedom to make life choices    156 non-null float64
Generosity                      156 non-null float64
Perceptions of corruption       156 non-null float64
dtypes: float64(7), int64(1), object(1)
memory usage: 11.0+ KB


### Compétence C2 : Nettoyage des données à importer

In [35]:
# standardize happiness2017's column names with the other years
mapping2017 = {'Happiness.Rank': 'Happiness Rank', 'Happiness.Score': 'Happiness Score', 'Economy..GDP.per.Capita.':'Economy (GDP per Capita)', 'Health..Life.Expectancy.':'Health (Life Expectancy)', 'Trust..Government.Corruption.': 'Trust (Government Corruption)','Dystopia.Residual':'Dystopia Residual'}
wh17 = wh17.rename(mapping2017, axis=1)

In [36]:
wh17.head()

,Country,Happiness Rank,Happiness Score,Whisker.high,Whisker.low,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Generosity,Trust (Government Corruption),Dystopia Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [37]:
# standardize happiness2018 and happiness2019 column names with the other years
mapping2018 = {'Overall rank': 'Happiness Rank', 'Country or region': 'Country', 'Score': 'Happiness Score', 'GDP per capita':'Economy (GDP per Capita)', 'Social support':'Family','Healthy life expectancy':'Health (Life Expectancy)', 'Freedom to make life choices': 'Freedom','Perceptions of corruption': 'Trust (Government Corruption)'}
wh18 = wh18.rename(mapping2018, axis=1)
wh19 = wh19.rename(mapping2018, axis=1)

In [38]:
wh19.head()

,Happiness Rank,Country,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Generosity,Trust (Government Corruption)
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


In [39]:
# simplify the column names
mapping = {'Economy (GDP per Capita)': 'Economy', 'Health (Life Expectancy)': 'Health', 'Trust (Government Corruption)': 'Trust' }
wh15 = wh15.rename(mapping, axis=1)
wh16 = wh16.rename(mapping, axis=1)
wh17 = wh17.rename(mapping, axis=1)
wh18 = wh18.rename(mapping, axis=1)
wh19 = wh19.rename(mapping, axis=1)

In [40]:
# removes extra spaces in column names
wh15.columns = wh15.columns.str.strip().str.replace(' ', '').str.replace('_', '').str.replace('(', '').str.replace(')', '')
wh16.columns = wh16.columns.str.strip().str.replace(' ', '').str.replace('_', '').str.replace('(', '').str.replace(')', '')
wh17.columns = wh17.columns.str.strip().str.replace(' ', '').str.replace('_', '').str.replace('(', '').str.replace(')', '')
wh18.columns = wh18.columns.str.strip().str.replace(' ', '').str.replace('_', '').str.replace('(', '').str.replace(')', '')
wh19.columns = wh19.columns.str.strip().str.replace(' ', '').str.replace('_', '').str.replace('(', '').str.replace(')', '')

In [41]:
wh19.head()

,HappinessRank,Country,HappinessScore,Economy,Family,Health,Freedom,Generosity,Trust
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298


In [42]:
# add the missing 'DystopiaResidual' column to the source files
wh18['DystopiaResidual'] = wh18['HappinessScore'] - (wh18['Economy'] + wh18['Family'] + wh18['Health'] + wh18['Freedom'] + wh18['Generosity'] + wh18['Trust'])
wh19['DystopiaResidual'] = wh19['HappinessScore'] - (wh19['Economy'] + wh19['Family'] + wh19['Health'] + wh19['Freedom'] + wh19['Generosity'] + wh19['Trust'])

In [43]:
# select the columns
cols_whr = ['Country', 'HappinessRank', 'HappinessScore', 'Economy', 'Family', 'Health', 'Freedom', 
            'Generosity', 'Trust', 'DystopiaResidual']
wh15 = wh15[cols_whr]
wh16 = wh16[cols_whr]
wh17 = wh17[cols_whr]
wh18 = wh18[cols_whr]
wh19 = wh19[cols_whr]

In [44]:
# add column 'Years' in preparation of a merge
wh15.insert(0, 'Years', 2015)
wh16.insert(0, 'Years', 2016)
wh17.insert(0, 'Years', 2017)
wh18.insert(0, 'Years', 2018)
wh19.insert(0, 'Years', 2019)

In [45]:
# get ready to a jointure with Countries
wh15['Country'] = wh15['Country'].str.lower()
wh16['Country'] = wh16['Country'].str.lower()
wh17['Country'] = wh17['Country'].str.lower()
wh18['Country'] = wh18['Country'].str.lower()
wh19['Country'] = wh19['Country'].str.lower()

In [46]:
whr = pd.concat([wh15, wh16, wh17, wh18, wh19], ignore_index=True)

In [47]:
wh18.head()

,Years,Country,HappinessRank,HappinessScore,Economy,Family,Health,Freedom,Generosity,Trust,DystopiaResidual
0,2018,finland,1,7.632,1.305,1.592,0.874,0.681,0.202,0.393,2.585
1,2018,norway,2,7.594,1.456,1.582,0.861,0.686,0.286,0.340,2.383
2,2018,denmark,3,7.555,1.351,1.590,0.868,0.683,0.284,0.408,2.371
3,2018,iceland,4,7.495,1.343,1.644,0.914,0.677,0.353,0.138,2.426
4,2018,switzerland,5,7.487,1.420,1.549,0.927,0.660,0.256,0.357,2.318


### Data cleaning

In [48]:
# identify the column names of World Happiness to match with World Bank's columns
wh_countries = whr['Country'].unique()
delta_c = sorted((diff_s(list(wh_countries), wdc['Country'])))
delta_c

['congo (brazzaville)',
 'congo (kinshasa)',
 'gambia',
 'hong kong',
 'hong kong s.a.r., china',
 'ivory coast',
 'kyrgyzstan',
 'laos',
 'macedonia',
 'north cyprus',
 'northern cyprus',
 'palestinian territories',
 'slovakia',
 'somaliland region',
 'south korea',
 'swaziland',
 'syria',
 'taiwan',
 'taiwan province of china',
 'trinidad & tobago']

In [49]:
# remove World Happiness countries that have no matches in World Bank
excluded_countries = ['north cyprus', 'gambia', 'northern cyprus', 'trinidad & tobago', 
                      'palestinian territories','somaliland region','taiwan', 'taiwan province of china']
whr = whr[~whr["Country"].isin(excluded_countries)]

In [50]:
# in case of discrepancies, chose World Bank over World Happiness
mapping = {"congo (brazzaville)": "congo", "congo (kinshasa)": "dem. rep. congo", 
           "hong kong": "hong kong sar, china", "hong kong s.a.r., china": "hong kong sar, china", 
           "ivory coast": "côte d'ivoire", "kyrgyzstan": "kyrgyz republic", "laos": "lao pdr", 
           "macedonia": "north macedonia", "slovakia": "slovak republic", "south korea": "korea",
            "swaziland": "eswatini", "syria": "syrian arab republic"}

whr = whr.replace({"Country": mapping})

In [51]:
# check that there is no discrepancies between World Happiness and World Bank
wh_countries = whr['Country'].unique()
delta_c = sorted((diff_s(list(wh_countries), wdc['Country'])))
delta_c

[]

### Compétence C4 : Recherche automatique ou manuelle des erreurs

In [52]:
# add the 'CountryCode' column for a coming jointure with Country
whr['CountryCode'] = whr['Country'].map(wdc.set_index('Country')['CountryCode'])

In [53]:
whr.shape

(762, 12)

In [54]:
whr.head()

,Years,Country,HappinessRank,HappinessScore,Economy,Family,Health,Freedom,Generosity,Trust,DystopiaResidual,CountryCode
0,2015,switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.29678,0.41978,2.51738,CHE
1,2015,iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.43630,0.14145,2.70201,ISL
2,2015,denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.34139,0.48357,2.49204,DNK
3,2015,norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.34699,0.36503,2.46531,NOR
4,2015,canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.45811,0.32957,2.45176,CAN


In [55]:
whr.tail()

,Years,Country,HappinessRank,HappinessScore,Economy,Family,Health,Freedom,Generosity,Trust,DystopiaResidual,CountryCode
777,2019,rwanda,152,3.334,0.359,0.711,0.614,0.555,0.217,0.411,0.467,RWA
778,2019,tanzania,153,3.231,0.476,0.885,0.499,0.417,0.276,0.147,0.531,TZA
779,2019,afghanistan,154,3.203,0.350,0.517,0.361,0.000,0.158,0.025,1.792,AFG
780,2019,central african republic,155,3.083,0.026,0.000,0.105,0.225,0.235,0.035,2.457,CAF
781,2019,south sudan,156,2.853,0.306,0.575,0.295,0.010,0.202,0.091,1.374,SSD


### Creation of data backup files

In [56]:
wdc.to_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\db\backup\wdc.csv', index = None, header=True)
wdi.to_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\db\backup\wdi.csv', index = None, header=True) 
whr.to_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\db\backup\whr.csv', index = None, header=True) 

connection sans MySQL. A enlever si RunAll
wdc = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\db\backup\wdc.csv')
wdi = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\db\backup\wdi.csv')
whr = pd.read_csv(r'C:\WPy64-3740\notebooks\Projets\world_happiness\db\backup\whr.csv')

### Create the database

In [57]:
pwd = getpass.getpass()

········


In [58]:
connect_engine='mysql+pymysql://root:' + pwd \
+ '@localhost/whr_huy?host=localhost?port=3306'

In [59]:
# connect to MySQL
# the db whr_huy has been created beforhand
engine = db.create_engine(connect_engine)
connection = engine.connect()

In [60]:
# delete the db 
engine.execute('drop schema if exists whr_huy')
engine.execute('create database whr_huy')
engine.execute('USE whr_huy')

In [61]:
# import 3 DF into 3 tables
wdc.to_sql('countries', connection, if_exists='replace', index=False)
wdi.to_sql('indicators', connection, if_exists='replace', index=False)
whr.to_sql('whr', connection, if_exists='replace', index=False)

In [62]:
if connection:                        
    connection.close() 

### Measure column sizes to optimise the dimensioning of SQL tables

In [63]:
measurer = np.vectorize(len)

In [64]:
res_wdc = measurer(wdc.values.astype(str)).max(axis=0)
res_wdc

array([ 3, 30, 26, 19,  5])

In [65]:
res_wdi = measurer(wdi.values.astype(str)).max(axis=0)
res_wdi

array([30,  3, 93, 17,  4, 21])

In [66]:
res_whr = measurer(whr.values.astype(str)).max(axis=0)
res_whr

array([ 4, 24,  3, 18, 20, 19, 20, 20, 20, 21, 19,  3])